In [1]:
import os
import pandas as pd

In [22]:
from io import StringIO
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient

from src.utils.cleaning import format_to_decimal

ModuleNotFoundError: No module named 'src'

In [3]:
service_client = BlobServiceClient.from_connection_string(os.getenv("AZURE_CONNECTION_STRING"))
blob_container = service_client.get_container_client(os.getenv("CONTAINER_NAME"))

In [18]:
def clean_and_add_features(df: pd.DataFrame, source: str):

    df.insert(0, "Source", source)
    df["Volume"] = df["Volume"].str.replace(',','').replace("'","").replace(".","").astype(int)
    df["Date"] = pd.to_datetime(df["Date"], format='%m/%d/%Y')
    
    df["Open"] = df["Open"].apply(format_to_decimal)
    df["High"] = df["High"].apply(format_to_decimal)
    df["Close"] = df["Close"].apply(format_to_decimal)
    df["Low"] = df["Low"].apply(format_to_decimal)

    # Dates
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    df["Year"] = df["Date"].dt.year
    df["DayOfWeek"] = df["Date"].dt.dayofweek

    df["MA_5"] = df["Close"].rolling(5).mean()
    df["MA_20"] = df["Close"].rolling(20).mean()
    df["EMA_5"] = df["Close"].ewm(span=5, adjust=False).mean()
    df["EMA_20"] = df["Close"].ewm(span=20, adjust=False).mean()
    df["STD_5"] = df["Close"].rolling(5).mean()
    df["STD_20"] = df["Close"].rolling(20).mean()

    df["DailyReturn"] = df["Close"].pct_change()
    df["Volatility"] = df["DailyReturn"].rolling(20).std()
    df["PriceChange"] = df["Close"] - df["Open"]
 
    return df

In [19]:
def upload_silver_to_blob(df, serviceClient, containerName, blobName):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    csv_buffer.seek(0)

    #blob_service_client = BlobServiceClient.from_connection_string(os.getenv("AZURE_CONNECTION_STRING"))
    blob_client = serviceClient.get_blob_client(container=containerName, blob=blobName)

    blob_client.upload_blob(csv_buffer.getvalue(), overwrite=True)


In [20]:
def main():
    print("Starting main...")
    load_dotenv()

    serviceClient = BlobServiceClient.from_connection_string(os.getenv("AZURE_CONNECTION_STRING"))
    blobContainer = serviceClient.get_container_client(os.getenv("CONTAINER_NAME"))

    bronzeLocation = os.getenv("BRONZE_LOCATION")
    silverFileOutput = os.getenv("SILVER_OUTPUT")
    print(bronzeLocation)
    for blob in blob_container.list_blobs(name_starts_with=f"{bronzeLocation}"):
        print(blob.name)
        blob_client = blobContainer.get_blob_client(blob.name)
        content = blob_client.download_blob().readall().decode('utf-8')

        file = pd.read_csv(StringIO(content))
        final = clean_and_add_features(file, os.path.basename(blob.name).replace('.csv',''))


        #upload_silver_to_blob(final, serviceClient, blobContainer, f"{silverFileOutput}silver_output.csv")

In [21]:
main()

Starting main...
bronze/
bronze/bdo.csv


NameError: name 'format_to_decimal' is not defined

In [16]:
final

NameError: name 'final' is not defined

In [17]:
file

NameError: name 'file' is not defined